In [5]:
import cv2
import numpy as np

video_path = "low_res_clip.mp4"  
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)

kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32)

kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

tracking = False
predicted_center = None

subtractor = cv2.createBackgroundSubtractorMOG2(history=0, varThreshold=100, detectShadows=True)

output_path = "volleyball_tracking_8connected.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (640, 480))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mean_val = np.mean(gray)
    std_val = np.std(gray)
    dynamic_threshold = int(mean_val + 0.2 * std_val)
    _, thresholded = cv2.threshold(gray, dynamic_threshold, 255, cv2.THRESH_TOZERO)
    
    thresholded = subtractor.apply(thresholded)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closed_image = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed_image, connectivity=8)
    
    min_area = 30  
    max_area = 50  
    
    detected_ball = None
    
    for i in range(1, num_labels):  
        area = stats[i, cv2.CC_STAT_AREA]
        
        if min_area <= area <= max_area:
            x, y, w, h = stats[i, cv2.CC_STAT_LEFT], stats[i, cv2.CC_STAT_TOP], stats[i, cv2.CC_STAT_WIDTH], stats[i, cv2.CC_STAT_HEIGHT]
            center_x = x + w // 2
            center_y = y + h // 2
            detected_ball = (center_x, center_y)
            radius = int(1 * (w + h) / 2)
            cv2.circle(frame, detected_ball, radius, (255, 0, 0), 2)
            break

    if detected_ball:
        tracking = True
        kalman.correct(np.array([[np.float32(detected_ball[0])], [np.float32(detected_ball[1])]]))

    prediction = kalman.predict()
    predicted_center = (int(prediction[0]), int(prediction[1]))

    # if tracking:
    #     cv2.circle(frame, predicted_center, 10, (0, 0, 255), 2)

    out.write(frame)

    cv2.imshow("Volleyball Detection | Thresholding + Kalman Tracking", frame)
    cv2.imshow("Thresholded Image", thresholded)
    cv2.imshow("Closed Image", closed_image)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
out.release()  
cv2.destroyAllWindows()


C:\Users\harjo\AppData\Local\Temp\ipykernel_35764\2710187053.py:71: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_center = (int(prediction[0]), int(prediction[1]))
